In [ ]:
pip install langchain-community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path="/Users/sabihakhan/Downloads/nke-10k-2023.pdf" 
loader = PyPDFLoader(file_path)
docs = loader.load()
#print(len(docs))
print(docs[0])

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
print(len(all_splits))

In [ ]:
pip install -qU langchain-core

In [ ]:
import os
impor

if not os.environ.get("OPENAI_API_KEY"): 
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter api key for OPENAI: ")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter api key for Langsmith: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents=all_splits)

assert len(vector_1) == len(vector_2)
print(f"Generated vector of length {len(vector_1)}")
print(f"GENERATED VECTOR -{vector_1[:10]}")

from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query:str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

retriever.batch([
    "How many distribution centers does Nike have in the US?",
    "When was Nike incorporated?",
])